In [1]:
import pandas as pd
import re

# ========================================
# 实验：先查重 vs 先清洗再查重
# ========================================

print("=" * 60)
print("🔬 实验：证明应该先清洗再查重")
print("=" * 60)

# 原始数据（包含格式噪声）
data_raw = [
    "Ordinateur portable HP",
    "ORDINATEUR PORTABLE HP",
    "<b>Ordinateur portable HP</b>"
]

df_raw = pd.DataFrame({"produit": data_raw})

print("\n📦 原始数据：")
print(df_raw)

# ----------------------------------------
# 方法 A：直接在原始数据上查重
# ----------------------------------------
print("\n" + "=" * 60)
print("❌ 方法 A：先查重（在脏数据上）")
print("=" * 60)

duplicates_before = df_raw.duplicated()
print(f"\n结果：发现 {duplicates_before.sum()} 个重复项")
print("\n详细标记：")
print(pd.DataFrame({
    "produit": df_raw["produit"],
    "est_doublon": duplicates_before
}))

print("\n⚠️  问题：三条语义相同的记录被判定为不同！")

# ----------------------------------------
# 方法 B：先清洗，再查重
# ----------------------------------------
print("\n" + "=" * 60)
print("✅ 方法 B：先清洗，再查重")
print("=" * 60)

def nettoyer_texte(texte):
    """清洗文本：去除HTML、转小写、去多余空格"""
    # 去除HTML标签
    texte = re.sub(r'<[^>]+>', '', texte)
    # 转小写
    texte = texte.lower()
    # 去除多余空格
    texte = ' '.join(texte.split())
    return texte

# 应用清洗
df_clean = df_raw.copy()
df_clean["produit_nettoye"] = df_clean["produit"].apply(nettoyer_texte)

print("\n🧹 清洗后的数据：")
print(df_clean)

# 现在查重
duplicates_after = df_clean["produit_nettoye"].duplicated()
print(f"\n结果：发现 {duplicates_after.sum()} 个重复项")
print("\n详细标记：")
print(pd.DataFrame({
    "原始": df_raw["produit"],
    "清洗后": df_clean["produit_nettoye"],
    "est_doublon": duplicates_after
}))

print("\n✅ 成功：正确识别了 2 个重复项（保留第一条）")

# ----------------------------------------
# 对比总结
# ----------------------------------------
print("\n" + "=" * 60)
print("📊 对比总结")
print("=" * 60)
print(f"原始数据条数：         {len(df_raw)}")
print(f"方法A 识别的重复数：   {duplicates_before.sum()}")
print(f"方法B 识别的重复数：   {duplicates_after.sum()}")
print(f"\n差异：方法B 多识别了 {duplicates_after.sum() - duplicates_before.sum()} 个真实重复！")

# ----------------------------------------
# 最终数据集对比
# ----------------------------------------
print("\n" + "=" * 60)
print("🎯 最终保留的数据")
print("=" * 60)

print("\n方法A（先查重）会保留：")
df_method_a = df_raw[~duplicates_before]
print(df_method_a)
print(f"→ 保留了 {len(df_method_a)} 条记录（错误：包含重复）")

print("\n方法B（先清洗再查重）会保留：")
df_method_b = df_clean[~duplicates_after]
print(df_method_b[["produit", "produit_nettoye"]])
print(f"→ 保留了 {len(df_method_b)} 条记录（正确：无重复）")

print("\n" + "=" * 60)
print("💡 结论：必须先清洗标准化，再查找重复！")
print("=" * 60)


🔬 实验：证明应该先清洗再查重

📦 原始数据：
                         produit
0         Ordinateur portable HP
1         ORDINATEUR PORTABLE HP
2  <b>Ordinateur portable HP</b>

❌ 方法 A：先查重（在脏数据上）

结果：发现 0 个重复项

详细标记：
                         produit  est_doublon
0         Ordinateur portable HP        False
1         ORDINATEUR PORTABLE HP        False
2  <b>Ordinateur portable HP</b>        False

⚠️  问题：三条语义相同的记录被判定为不同！

✅ 方法 B：先清洗，再查重

🧹 清洗后的数据：
                         produit         produit_nettoye
0         Ordinateur portable HP  ordinateur portable hp
1         ORDINATEUR PORTABLE HP  ordinateur portable hp
2  <b>Ordinateur portable HP</b>  ordinateur portable hp

结果：发现 2 个重复项

详细标记：
                              原始                     清洗后  est_doublon
0         Ordinateur portable HP  ordinateur portable hp        False
1         ORDINATEUR PORTABLE HP  ordinateur portable hp         True
2  <b>Ordinateur portable HP</b>  ordinateur portable hp         True

✅ 成功：正确识别了 2 个重复项（保留第一条）

📊 对比总结
原始数据

In [2]:
import pandas as pd

# ========================================
# 实验：duplicated() 是否区分大小写？
# ========================================

print("=" * 60)
print("🔬 测试：duplicated() 对大小写的行为")
print("=" * 60)

# 创建测试数据
data = [
    "Apple",
    "apple",
    "APPLE",
    "Apple"  # 这个和第一个完全相同
]

df = pd.DataFrame({"fruit": data})

print("\n📦 测试数据：")
print(df)

# 使用 duplicated() 检测
duplicates = df["fruit"].duplicated()

print("\n🔍 duplicated() 的判断结果：")
result_df = pd.DataFrame({
    "fruit": df["fruit"],
    "是否重复": duplicates,
    "说明": ["首次出现", "不同（小写）", "不同（大写）", "重复（完全相同）"]
})
print(result_df)

print("\n" + "=" * 60)
print(f"✅ 结论：duplicated() 识别了 {duplicates.sum()} 个重复")
print(f"   → 只有第4行（'Apple'）被识别为重复")
print(f"   → 'Apple' vs 'apple' vs 'APPLE' 被视为不同")
print("=" * 60)

# ----------------------------------------
# 对比：如果先标准化呢？
# ----------------------------------------
print("\n" + "=" * 60)
print("🧹 对比：先转小写再检测")
print("=" * 60)

df["fruit_lower"] = df["fruit"].str.lower()
duplicates_after = df["fruit_lower"].duplicated()

print("\n📊 标准化后的结果：")
result_df2 = pd.DataFrame({
    "原始": df["fruit"],
    "标准化": df["fruit_lower"],
    "是否重复": duplicates_after
})
print(result_df2)

print(f"\n✅ 标准化后识别了 {duplicates_after.sum()} 个重复")
print(f"   → 'apple', 'APPLE', 'Apple'(第4个) 都被识别为重复")

# ----------------------------------------
# 更详细的对比
# ----------------------------------------
print("\n" + "=" * 60)
print("📈 数据对比")
print("=" * 60)
print(f"原始数据条数：              {len(df)}")
print(f"方法1（直接duplicated）：   识别 {duplicates.sum()} 个重复")
print(f"方法2（先lower再检测）：    识别 {duplicates_after.sum()} 个重复")
print(f"\n差异：方法2 多识别了 {duplicates_after.sum() - duplicates.sum()} 个语义重复！")

🔬 测试：duplicated() 对大小写的行为

📦 测试数据：
   fruit
0  Apple
1  apple
2  APPLE
3  Apple

🔍 duplicated() 的判断结果：
   fruit   是否重复        说明
0  Apple  False      首次出现
1  apple  False    不同（小写）
2  APPLE  False    不同（大写）
3  Apple   True  重复（完全相同）

✅ 结论：duplicated() 识别了 1 个重复
   → 只有第4行（'Apple'）被识别为重复
   → 'Apple' vs 'apple' vs 'APPLE' 被视为不同

🧹 对比：先转小写再检测

📊 标准化后的结果：
      原始    标准化   是否重复
0  Apple  apple  False
1  apple  apple   True
2  APPLE  apple   True
3  Apple  apple   True

✅ 标准化后识别了 3 个重复
   → 'apple', 'APPLE', 'Apple'(第4个) 都被识别为重复

📈 数据对比
原始数据条数：              4
方法1（直接duplicated）：   识别 1 个重复
方法2（先lower再检测）：    识别 3 个重复

差异：方法2 多识别了 2 个语义重复！


In [6]:
import pandas as pd
import re

# ========================================
# 实验：duplicated() 如何处理换行符？
# ========================================

print("=" * 60)
print("🔬 测试：duplicated() 对换行符的行为")
print("=" * 60)

# 创建包含各种换行符的测试数据
data = [
    "Hello World",           # 普通文本
    "Hello\nWorld",          # \n (Unix/Linux 换行)
    "Hello\r\nWorld",        # \r\n (Windows 换行)
    "Hello\rWorld",          # \r (旧 Mac 换行)
    "Hello World",           # 和第一个完全相同
    "Hello  World",          # 双空格（不是换行）
]

df = pd.DataFrame({"text": data})

print("\n📦 原始数据（repr显示）：")
for idx, text in enumerate(data):
    print(f"{idx}: {repr(text)}")

print("\n" + "-" * 60)

# 使用 duplicated() 检测
duplicates = df["text"].duplicated()

print("\n🔍 duplicated() 的判断结果：")
result_df = pd.DataFrame({
    "索引": range(len(data)),
    "文本内容（repr）": [repr(t) for t in data],
    "是否重复": duplicates
})
print(result_df.to_string(index=False))

print("\n" + "=" * 60)
print(f"✅ 直接使用 duplicated()：识别了 {duplicates.sum()} 个重复")
print(f"   → 只有索引4（完全相同的字符串）被识别")
print(f"   → \\n, \\r\\n, \\r 都被视为不同的字符串！")
print("=" * 60)

# ----------------------------------------
# 对比：如果先标准化换行符呢？
# ----------------------------------------
print("\n" + "=" * 60)
print("🧹 对比：标准化换行符后再检测")
print("=" * 60)

# 方法1：统一替换为空格
df["normalized_space"] = df["text"].str.replace(r'[\r\n]+', ' ', regex=True)
duplicates_space = df["normalized_space"].duplicated()

print("\n方法1：换行符 → 空格")
result1 = pd.DataFrame({
    "原始": [repr(t) for t in data],
    "标准化": df["normalized_space"],
    "是否重复": duplicates_space
})
print(result1.to_string(index=False))
print(f"\n识别了 {duplicates_space.sum()} 个重复")

# 方法2：统一替换为 \n
df["normalized_newline"] = df["text"].str.replace(r'\r\n|\r', '\n', regex=True)
duplicates_newline = df["normalized_newline"].duplicated()

print("\n" + "-" * 60)
print("方法2：统一为 \\n")
result2 = pd.DataFrame({
    "原始": [repr(t) for t in data],
    "标准化": [repr(t) for t in df["normalized_newline"]],
    "是否重复": duplicates_newline
})
print(result2.to_string(index=False))
print(f"\n识别了 {duplicates_newline.sum()} 个重复")

# 方法3：完全去除换行符
df["no_newline"] = df["text"].str.replace(r'[\r\n\s]+', ' ', regex=True).str.strip()
duplicates_no_newline = df["no_newline"].duplicated()

print("\n" + "-" * 60)
print("方法3：去除所有空白字符（包括换行和多余空格）")
result3 = pd.DataFrame({
    "原始": [repr(t) for t in data],
    "标准化": df["no_newline"],
    "是否重复": duplicates_no_newline
})
print(result3.to_string(index=False))
print(f"\n识别了 {duplicates_no_newline.sum()} 个重复")

# ----------------------------------------
# 总结对比
# ----------------------------------------
print("\n" + "=" * 60)
print("📊 方法对比总结")
print("=" * 60)
comparison = pd.DataFrame({
    "方法": [
        "直接 duplicated()",
        "方法1: 换行→空格",
        "方法2: 统一为\\n",
        "方法3: 去除所有空白"
    ],
    "识别的重复数": [
        duplicates.sum(),
        duplicates_space.sum(),
        duplicates_newline.sum(),
        duplicates_no_newline.sum()
    ],
    "说明": [
        "严格字符比较",
        "保留单词分隔",
        "保留换行语义",
        "最激进的清洗"
    ]
})
print(comparison.to_string(index=False))

print("\n💡 结论：换行符（\\n, \\r\\n, \\r）在 duplicated() 眼中")
print("   都是不同的字符，必须先标准化！")

🔬 测试：duplicated() 对换行符的行为

📦 原始数据（repr显示）：
0: 'Hello World'
1: 'Hello\nWorld'
2: 'Hello\r\nWorld'
3: 'Hello\rWorld'
4: 'Hello World'
5: 'Hello  World'

------------------------------------------------------------

🔍 duplicated() 的判断结果：
 索引       文本内容（repr）  是否重复
  0    'Hello World' False
  1   'Hello\nWorld' False
  2 'Hello\r\nWorld' False
  3   'Hello\rWorld' False
  4    'Hello World'  True
  5   'Hello  World' False

✅ 直接使用 duplicated()：识别了 1 个重复
   → 只有索引4（完全相同的字符串）被识别
   → \n, \r\n, \r 都被视为不同的字符串！

🧹 对比：标准化换行符后再检测

方法1：换行符 → 空格
              原始          标准化  是否重复
   'Hello World'  Hello World False
  'Hello\nWorld'  Hello World  True
'Hello\r\nWorld'  Hello World  True
  'Hello\rWorld'  Hello World  True
   'Hello World'  Hello World  True
  'Hello  World' Hello  World False

识别了 4 个重复

------------------------------------------------------------
方法2：统一为 \n
              原始            标准化  是否重复
   'Hello World'  'Hello World' False
  'Hello\nWorld' 'Hello\nWorld' False
'Hello\r\

In [ ]:
COL_ID = "productid"

mask = df["__title_clean"].ne("") & df["__desc_clean"].ne("")
sub  = df.loc[mask, ["__title_clean","__desc_clean", COL_ID, "image_path"]].copy()

# 组内大小
sub["_pair_count"] = sub.groupby(["__title_clean","__desc_clean"])[COL_ID].transform("size")
dupe_both = sub.loc[sub["_pair_count"] >= 2].copy()

# ---- 2) 组级聚合：count、ids、images(唯一且保序) ----
import pandas as pd

def _unique_keep_order(seq):
    seen, out = set(), []
    for v in seq:
        if pd.isna(v) or v in (None, ""):
            continue
        if v not in seen:
            seen.add(v); out.append(v)
    return out

summary = (
    dupe_both
    .groupby(["__title_clean","__desc_clean"], dropna=False)
    .agg(
        count = (COL_ID, "count"),
        ids   = (COL_ID, lambda x: list(x)),
        images= ("image_path", lambda x: _unique_keep_order(list(x)))
    )
    .sort_values("count", ascending=False)
)

# ---- 3) 总数统计 ----
total_groups  = len(summary)                 # “标题+描述相同”的组数
total_records = int(summary["count"].sum())  # 涉及的记录总条数
print(f"两键都相同：{total_groups} 组，共 {total_records} 条记录。")

# ---- 4) 显示前5组：把图片都放在最后一格（用你的 img_cell() 转 data URI）----
from IPython.display import display, HTML

def show_groups_table_with_images(top_n=5, max_imgs_per_group=8, thumb_size=128):
    rows_html = []
    tops = summary.head(top_n).reset_index()

    for _, r in tops.iterrows():
        t_clean = r["__title_clean"]
        d_clean = r["__desc_clean"]
        title_preview = (t_clean[:140] + "…") if len(t_clean) > 140 else t_clean
        desc_preview  = (d_clean[:140] + "…") if len(d_clean) > 140 else d_clean

        ids_sample = ", ".join(map(str, r["ids"][:10])) + (" ..." if len(r["ids"]) > 10 else "")

        # 组内图片（唯一集合中取前 max_imgs_per_group 张）
        img_tags = []
        for p in (r["images"][:max_imgs_per_group] if isinstance(r["images"], list) else []):
            tag = img_cell(p, size=thumb_size)  # 复用你的函数：本地->base64 data URI
            img_tags.append(tag if tag else '<div style="display:inline-block;width:'
                                 f'{thumb_size}px;height:{thumb_size}px;border:1px dashed #ccc;'
                                 'border-radius:6px;line-height:{thumb_size}px;text-align:center;color:#888;">no img</div>')
        imgs_html = "".join(img_tags) if img_tags else (
            f'<div style="display:inline-block;width:{thumb_size}px;height:{thumb_size}px;'
            'border:1px dashed #ccc;border-radius:6px;line-height:{thumb_size}px;'
            'text-align:center;color:#888;">no img</div>'
        )

        row = f"""
        <tr>
          <td style="vertical-align:top;"><code>{r['count']}</code></td>
          <td style="vertical-align:top;">{title_preview}</td>
          <td style="vertical-align:top;">{desc_preview}</td>
          <td style="vertical-align:top;white-space:nowrap;">{ids_sample}</td>
          <td style="vertical-align:top;">{imgs_html}</td>
        </tr>
        """
        rows_html.append(row)

    html = f"""
    <div style="margin:10px 0;">
      <div style="margin-bottom:8px;font-weight:600;">
        前 {top_n} 组（标题&描述相同）。总计：{total_groups} 组 / {total_records} 条记录
      </div>
      <table style="width:100%;border-collapse:collapse;">
        <thead>
          <tr>
            <th style="text-align:left;border-bottom:1px solid #ddd;padding:6px;">count</th>
            <th style="text-align:left;border-bottom:1px solid #ddd;padding:6px;">title (clean)</th>
            <th style="text-align:left;border-bottom:1px solid #ddd;padding:6px;">desc (clean)</th>
            <th style="text-align:left;border-bottom:1px solid #ddd;padding:6px;">IDs (sample)</th>
            <th style="text-align:left;border-bottom:1px solid #ddd;padding:6px;">images</th>
          </tr>
        </thead>
        <tbody>
          {''.join(rows_html)}
        </tbody>
      </table>
    </div>
    """
    display(HTML(html))

# 调用：显示前5组，每组最多8张缩略图（128px）
show_groups_table_with_images(top_n=5, max_imgs_per_group=8, thumb_size=128)

